### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.2/886.2 kB 11.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.187.0-py2.py3-none-any.whl size=1186949 sha256=dfb23529462fe2921318f64d7df7fd36e6fe441ab0528d7b24a556040fcb4253
  Stored in directory: /home/ec2-user/.cache/pip/wheels/21/d2/2b/ed240a9d3084c5b4e3e7fd2ca8f9c11659bb98f0b87b6b1ca3
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.184.0
    Uninstalling sagemaker-2.184.0:
      Successfully uninstalled sagemaker-2.184.0


In [2]:
!rm -rf ./LLM_sqlcoder_model

In [9]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_sqlcoder_model")
local_model_path.mkdir(exist_ok=True)
model_name = "defog/sqlcoder"
#commit_hash = "9e3ca34b26ace9f0c4e098f61f6fa8e9d89dc2c2"

* 如果已经下载该步骤不用再执行

In [4]:
snapshot_download(repo_id=model_name, cache_dir=local_model_path)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

'LLM_sqlcoder_model/models--defog--sqlcoder/snapshots/bef4bcd10cc551fb3359a2ebc8f394f5e567f540'

### 2. 把模型拷贝到S3为后续部署做准备

In [10]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
s3_model_prefix = "LLM-RAG/workshop/LLM_sqlcoder_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_sqlcoder_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_sqlcoder_deploy_code
model_snapshot_path: LLM_sqlcoder_model/models--defog--sqlcoder/snapshots/bef4bcd10cc551fb3359a2ebc8f394f5e567f540


* 如果模型已经上传至S3该步骤不用执行

In [ ]:
!aws s3 rm --recursive s3://{bucket}/{s3_model_prefix}
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [13]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [14]:
!mkdir -p LLM_sqlcoder_deploy_code

In [43]:
%%writefile LLM_sqlcoder_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers
import torch
import deepspeed


def load_model(properties):
    tensor_parallel_degree = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location,trust_remote_code=True)
    
    tokenizer = AutoTokenizer.from_pretrained(model_location)
    model = AutoModelForCausalLM.from_pretrained(
      model_location,
      trust_remote_code=True,
      #torch_dtype=torch.float16,
      # torch_dtype=torch.bfloat16,
      #load_in_8bit=True,
      load_in_4bit=True,
      device_map="auto",
      use_cache=True,
    )
    #model = deepspeed.init_inference(model,
    #      tensor_parallel={"tp_size": tensor_parallel_degree},
    #      dtype=model.dtype,
    #      replace_method='auto',
    #      replace_with_kernel_inject=True)
    #
    #local_rank = int(os.getenv("LOCAL_RANK", "0"))
    #generator = pipeline(
    #    task="text-generation", model=model, tokenizer=tokenizer, device=local_rank, use_cache=True
    #)
    ### for accelerate ####
    generator = transformers.pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map="auto", use_cache=True)
    return model, generator, tokenizer


pipeline = None
tokenizer = None
model = None


def handle(inputs: Input):
    global model,pipeline, tokenizer
    if not pipeline:
        model, pipeline, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()

    input_sentences = data["inputs"]
    params = data["parameters"]
    input_ids = tokenizer.encode(input_sentences, return_tensors='pt').to('cuda')

    
    eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
    with torch.no_grad():  
       output_ids = model.generate(
          input_ids,
          num_return_sequences=1,
          eos_token_id=eos_token_id,
          pad_token_id=eos_token_id,
          **params          
       )
    response=tokenizer.decode(output_ids[0], skip_special_tokens=True)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    print(response.split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")
    result = {"outputs": response}
    return Output().add_as_json(result)

Overwriting LLM_sqlcoder_deploy_code/model.py


In [44]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_sqlcoder_model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [45]:
%%writefile LLM_sqlcoder_deploy_code/serving.properties
engine=Python
#engine=DeepSpeed
option.entryPoint=model.py
option.load_in_4bit=TRUE
option.tensor_parallel_degree=4
option.s3url=s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_sqlcoder_model/

Overwriting LLM_sqlcoder_deploy_code/serving.properties


#### 注意: 必须把transformers升级，否则会报找不到codeLlamaTokenizer
如果是中国区建议添加国内的pip镜像,如下代码所示

%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1


In [46]:
%%writefile LLM_sqlcoder_deploy_code/requirements.txt
#-i https://pypi.tuna.tsinghua.edu.cn/simple
#transformers==4.30.2
git+https://github.com/huggingface/transformers

Overwriting LLM_sqlcoder_deploy_code/requirements.txt


In [47]:
!rm model.tar.gz
!cd LLM_sqlcoder_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_sqlcoder_deploy_code

LLM_sqlcoder_deploy_code/
LLM_sqlcoder_deploy_code/requirements.txt
LLM_sqlcoder_deploy_code/model.py
LLM_sqlcoder_deploy_code/serving.properties


In [48]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_sqlcoder_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [49]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"sqlcoder") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

sqlcoder-2023-09-24-06-31-09-959
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-west-2:687912291502:model/sqlcoder-2023-09-24-06-31-09-959


In [50]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.12xlarge",
            #"InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            #"ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/sqlcoder-2023-09-24-06-31-09-959-config',
 'ResponseMetadata': {'RequestId': 'cc3fe13c-2eb4-4e06-b1a9-1e49eb9041ba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cc3fe13c-2eb4-4e06-b1a9-1e49eb9041ba',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '120',
   'date': 'Sun, 24 Sep 2023 06:31:10 GMT'},
  'RetryAttempts': 0}}

In [51]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:687912291502:endpoint/sqlcoder-2023-09-24-06-31-09-959-endpoint


#### 持续检测模型部署进度

In [52]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:687912291502:endpoint/sqlcoder-2023-09-24-06-31-09-959-endpoint
Status: InService


### 5. 模型测试

In [53]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_new_tokens": 400,
  #"temperature": 0.5,
  #"num_beams": 5, 
  "do_sample": False
}

CPU times: user 2.82 ms, sys: 0 ns, total: 2.82 ms
Wall time: 2.6 ms


In [54]:
question="I want to know the top bill customer's address"
sql_prompt= """
### Instructions:
Your task is to convert a question into a SQL query, given a Mysql database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the question `{question}`.
This query will run on a database whose schema is represented in this string:
CREATE TABLE customer (
	c_customer_sk INTEGER NOT NULL, 
	c_customer_id CHAR(16) NOT NULL, 
	c_current_cdemo_sk INTEGER, 
	c_current_hdemo_sk INTEGER, 
	c_current_addr_sk INTEGER, 
	c_first_shipto_date_sk INTEGER, 
	c_first_sales_date_sk INTEGER, 
	c_salutation CHAR(10), 
	c_first_name CHAR(20), 
	c_last_name CHAR(30), 
	c_preferred_cust_flag CHAR(1), 
	c_birth_day INTEGER, 
	c_birth_month INTEGER, 
	c_birth_year INTEGER, 
	c_birth_country VARCHAR(20), 
	c_login CHAR(13), 
	c_email_address CHAR(50), 
	c_last_review_date CHAR(10), 
	PRIMARY KEY (c_customer_sk)
)DEFAULT CHARSET=utf8 ENGINE=InnoDB


CREATE TABLE web_sales (
	ws_sold_date_sk INTEGER, 
	ws_sold_time_sk INTEGER, 
	ws_ship_date_sk INTEGER, 
	ws_item_sk INTEGER NOT NULL, 
	ws_bill_customer_sk INTEGER, 
	ws_bill_cdemo_sk INTEGER, 
	ws_bill_hdemo_sk INTEGER, 
	ws_bill_addr_sk INTEGER, 
	ws_ship_customer_sk INTEGER, 
	ws_ship_cdemo_sk INTEGER, 
	ws_ship_hdemo_sk INTEGER, 
	ws_ship_addr_sk INTEGER, 
	ws_web_page_sk INTEGER, 
	ws_web_site_sk INTEGER, 
	ws_ship_mode_sk INTEGER, 
	ws_warehouse_sk INTEGER, 
	ws_promo_sk INTEGER, 
	ws_order_number INTEGER NOT NULL, 
	ws_quantity INTEGER, 
	ws_wholesale_cost DECIMAL(7, 2), 
	ws_list_price DECIMAL(7, 2), 
	ws_sales_price DECIMAL(7, 2), 
	ws_ext_discount_amt DECIMAL(7, 2), 
	ws_ext_sales_price DECIMAL(7, 2), 
	ws_ext_wholesale_cost DECIMAL(7, 2), 
	ws_ext_list_price DECIMAL(7, 2), 
	ws_ext_tax DECIMAL(7, 2), 
	ws_coupon_amt DECIMAL(7, 2), 
	ws_ext_ship_cost DECIMAL(7, 2), 
	ws_net_paid DECIMAL(7, 2), 
	ws_net_paid_inc_tax DECIMAL(7, 2), 
	ws_net_paid_inc_ship DECIMAL(7, 2), 
	ws_net_paid_inc_ship_tax DECIMAL(7, 2), 
	ws_net_profit DECIMAL(7, 2), 
	PRIMARY KEY (ws_item_sk, ws_order_number)
)DEFAULT CHARSET=utf8 ENGINE=InnoDB
### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
""".format(question=question)

question="最近一个月温度合格的派车单数量"
sql_prompt2="""
### Instructions:
You are a MySQL expert,create a syntactically correct MySQL query,given a MySQL database schema.
Adhere to these rules:
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float
- Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks () to denote them as delimited identifiers.
- Pay attention to use CURDATE() function to get the current date, if the question involves "today".
### Input:
Generate a SQL query that answers the question `{question}`.
This query will run on a database whose schema is represented in this string:
CREATE TABLE ads_bi_quality_monitor_shipping_detail (
  shipping_order_code varchar(100)  COMMENT '派车单编码',
  license_plate varchar(100)  COMMENT '车牌号',
  truck_type varchar(50)  COMMENT '车辆类型',
  tenant_id varchar(100)  COMMENT '租户编码',
  tenant_name varchar(200)  COMMENT '租户名称',
  father_company_code varchar(100)  COMMENT '分子公司编码',
  father_company_name varchar(200)  COMMENT '分子公司名称',
  father_company_short_name varchar(200)  COMMENT '分子公司简称',
  start_transport_time varchar(50)  COMMENT '运输出发时间',
  signing_time varchar(50)  COMMENT '派车单签收时间',
  plan_start_time varchar(50)  COMMENT '派车单计划取货时间',
  plan_end_time varchar(50)  COMMENT '派车单计划送达时间',
  frist_fence_time varchar(50)  COMMENT '第一次碰撞装货地电子围栏时间',
  leave_load_station_2km_time varchar(50)  COMMENT '离开最后一个装货地电子围栏2km的时间',
  frist_arrive_unload_time varchar(50)  COMMENT '首次到达卸货点电子围栏时间',
  transport_type varchar(10)  COMMENT '运输类型(干线/城配)',
  plan_mileage varchar(50)  COMMENT '计划里程',
  driver_accept_time varchar(50)  COMMENT '司机接单时间',
  driver_name varchar(50)  COMMENT '司机姓名',
  driver_phone varchar(50)  COMMENT '司机电话',
  driver_type varchar(10)  COMMENT '司机类型',
  transport_tenant_id varchar(100)  COMMENT '运力承运商id',
  transport_tenant_name varchar(200)  COMMENT '运力承运商名称',
  warm_area varchar(50)  COMMENT '温区信息',
  waybill_count int DEFAULT NULL COMMENT '运单数量',
  gps_device_list varchar(100)  COMMENT 'gps设备',
  gps_report_dot_num int DEFAULT NULL COMMENT 'gps上报点数',
  temp_substandard_min decimal(20,8) DEFAULT NULL COMMENT '温度不达标时长(分钟)',
  temp_substandard_min_n12 decimal(20,8) DEFAULT NULL COMMENT '温度不达标时长(分钟)_-12',
  temp_substandard_min_n16 decimal(20,8) DEFAULT NULL COMMENT '温度不达标时长(分钟)_-16',
  shipping_order_transport_min decimal(20,8) DEFAULT NULL COMMENT '派车单运输总时长(分钟)',
  shipping_order_temp_standard_rate decimal(20,8) DEFAULT NULL COMMENT '派车单温度达标率',
  shipping_order_temp_standard_rate_n12 decimal(20,8) DEFAULT NULL COMMENT '派车单温度达标率_-12',
  shipping_order_temp_standard_rate_n16 decimal(20,8) DEFAULT NULL COMMENT '派车单温度达标率_-16',
  dep_fence_match_num int DEFAULT NULL COMMENT '出发地电子围栏匹配数量',
  dep_total_num int DEFAULT NULL COMMENT '出发地总数量',
  dep_fence_match_rate decimal(20,8) DEFAULT NULL COMMENT '出发地电子围栏匹配率',
  des_fence_match_num int DEFAULT NULL COMMENT '目的地电子围栏匹配数量',
  des_total_num int DEFAULT NULL COMMENT '目的地总数量',
  des_fence_match_rate decimal(20,8) DEFAULT NULL COMMENT '目的地电子围栏匹配率',
  exception_shipping_order_type varchar(10)  COMMENT '异常派车单情况,0-非异常',
  settlement_tenant_id varchar(100)  COMMENT '结算主体租户id',
  settlement_tenant_name varchar(100)  COMMENT '结算主体租户名称',
  truck_ownership varchar(100)  COMMENT '车辆所有权(自有/外请/临时)',
  load_finish_time varchar(50)  COMMENT '派车单点击装货完成时间',
  last_leave_unload_time varchar(50)  COMMENT '最后离开卸货点时间',
  temp_right_tag varchar(20)  COMMENT '温度是否合格(合格/不合格/不参与评估)',
  warm_area_type varchar(20)  COMMENT '温区类型。常温/冷链/自定义区间',
  source_dt varchar(20)  COMMENT '派车单来源的增量表dt',
  cur_shipping_odr_cust text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci COMMENT '当前派车单对应的客户',
  is_app_operation varchar(10)  COMMENT '是否APP操作卡控',
  leave_load_station_time_app varchar(50)  COMMENT '离开第一个装货地时间_app',
  frist_arrive_unload_time_app varchar(50)  COMMENT '到达第一个卸货点时间_app',
  last_arrive_unload_time_app varchar(50)  COMMENT '到达最后一个卸货点时间_app',
  temp_calc_start_time varchar(50)  COMMENT '温度计算开始时间',
  temp_calc_end_time varchar(50)  COMMENT '温度计算结束时间',
  exception_shipping_order_type_desc varchar(100)  COMMENT '异常派车单情况描述',
  root_shipping_odr_cust text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci COMMENT '根派车单对应的客户',
  is_use_custom_temp_range varchar(10)  COMMENT '是否使用自定义温度范围',
  temp_eval_lowest int DEFAULT NULL COMMENT '温度考核最低值',
  temp_eval_highest int DEFAULT NULL COMMENT '温度考核最高值',
  is_gps_cover varchar(10)  COMMENT 'GPS是否覆盖',
  shipping_create_time varchar(50)  COMMENT '派车单创建时间',
  gps_device_list_plan varchar(100)  COMMENT 'GPS设备_预估',
  is_gps_cover_plan varchar(10)  COMMENT 'GPS是否覆盖_预估',
  first_station_is_on_time varchar(10)  COMMENT '首店是否准时',
  is_many_warm varchar(10)  COMMENT '是否多温区',
  many_temp_standard_rate decimal(20,8) DEFAULT NULL COMMENT '多温区温度达标率',
  lc_standard_ratio decimal(20,8) DEFAULT NULL COMMENT '冷藏温度达标率',
  ld_standard_ratio decimal(20,8) DEFAULT NULL COMMENT '冷冻温度达标率',
  probe_warm_list text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci COMMENT '探头温区',
  KEY idx_shipping_code (shipping_order_code),
  KEY idx_sdt (source_dt)
) ENGINE=InnoDB  COMMENT='BI看板_品质监控_派车单明细';
### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
""".format(question=question)



response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": sql_prompt2,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

result_sql=response_model['Body'].read().decode('utf8')
print(result_sql.split("```sql")[-1].split("```")[0].split(";")[0].strip().replace("\\n"," ") + ";")

 SELECT count(*) FROM   ads_bi_quality_monitor_shipping_detail WHERE  DATE_SUB(CURDATE(), INTERVAL 1 MONTH) <= DATE(signing_time)    AND temp_right_tag = '合格'    AND source_dt = '2020-04-01';


#### 清除模型Endpoint和config

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name codellama-2023-06-27-06-53-41-986-endpoint